In [133]:
import keras
import tensorflow as tf
from keras.datasets import reuters
from keras.preprocessing.text import Tokenizer
from keras.layers import BatchNormalization
from keras.models import Model
from keras.layers import Dense, Conv1D, Flatten, Dropout, Input
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np


In [103]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(path="reuters.npz",
                                                         num_words=None,
                                                         skip_top=0,
                                                         maxlen=None,
                                                         test_split=0.2,
                                                         seed=113,
                                                         start_char=1,
                                                         oov_char=2,
                                                         index_from=3)

In [104]:
#Look at a small part of the data
print('Training data: ', x_train[0])
print('Training label: ', y_train[0])
print('Length of training data', len(x_train))
print('Length of test data', len(x_test))

Training data:  [1, 27595, 28842, 8, 43, 10, 447, 5, 25, 207, 270, 5, 3095, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 4579, 1005, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 1245, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]
Training label:  3
Length of training data 8982
Length of test data 2246


In [106]:
word_index = reuters.get_word_index(path="reuters_word_index.json")
#Check to see what index the word sport is, a rudimentary test of the index loading
word_index["sport"]

13074

In [107]:
#The index is organized to look up the integer value, it would be better to look up the key
integer_word_index = dict([(value, key) for (key, value) in word_index.items()])

# Now we can search for the word that aligns to a certain key, we looked up sport before so lets check that index
print('The word at index 13074 is: ',integer_word_index[13074])

#how many different words are in the index
print('There are', len(integer_word_index)+1, 'words in the word index')

The word at index 13074 is:  sport
There are 30980 words in the word index


In [108]:
#Max words in an article
max_words = 10000
#46 labels
LABEL_DIMENSIONS = max(y_train)+1

tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')

y_train = keras.utils.to_categorical(y_train, LABEL_DIMENSIONS)
y_test = keras.utils.to_categorical(y_test, LABEL_DIMENSIONS)

print(x_train[0])
print(len(x_train[0]))

print(y_train[0])
print(len(y_train[0]))

[0. 1. 0. ... 0. 0. 0.]
10000
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
46


In [148]:
def build_model(keep_prob=0.5, optimizer='adam'):
    inputs = Input(shape=[10000,], name="input") 
    
    #Convolution 1
    conv1 = Dense(64, activation="relu", name="conv_1")(inputs)

    #Convolution 2
    conv2 = Dense(64,  activation="relu", name="conv_2")(conv1)
    
    #output
    prediction=Dense(46, activation="softmax", name ="softmax")(conv2)

    # finalize and compile
    model = Model(inputs=inputs, outputs=prediction)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])
    return model

In [149]:
def create_hyperparameters():
    batches=[10,20,30,40,50,60,70]
    optimizers = ['rmsprop','adam','adadelta']
    dropout=np.linspace(0.1,0.5,5)
    epochs = [5]
    return {"batch_size":batches, "optimizer":optimizers, "keep_prob":dropout, "epochs":epochs,}


In [150]:
model = KerasClassifier(build_fn=build_model, verbose=1)
hyperparameters=create_hyperparameters()

In [151]:
search = RandomizedSearchCV(estimator=model, param_distributions=hyperparameters, n_iter=10, n_jobs=1, cv=3, verbose=0)

In [152]:
search.fit(x_train,y_train)

Epoch 1/5
5988/5988 [==============================] - 3s 531us/step - loss: 1.6681 - acc: 0.6431
Epoch 2/5
5988/5988 [==============================] - 3s 457us/step - loss: 0.8855 - acc: 0.8043
Epoch 3/5
5988/5988 [==============================] - 3s 472us/step - loss: 0.6025 - acc: 0.8617
Epoch 4/5
5988/5988 [==============================] - 3s 476us/step - loss: 0.4271 - acc: 0.9061
Epoch 5/5
5988/5988 [==============================] - 1s 151us/step
Epoch 1/5
5988/5988 [==============================] - 3s 522us/step - loss: 1.6948 - acc: 0.6259
Epoch 2/5
5988/5988 [==============================] - 3s 462us/step - loss: 0.9346 - acc: 0.7887
Epoch 3/5
5988/5988 [==============================] - 3s 536us/step - loss: 0.6383 - acc: 0.8584
Epoch 4/5
5988/5988 [==============================] - 3s 507us/step - loss: 0.4576 - acc: 0.9010
Epoch 5/5
5988/5988 [==============================] - 1s 153us/step
Epoch 1/5
5988/5988 [==============================] - 3s 531us/step - loss: 1

5988/5988 [==============================] - 3s 528us/step - loss: 0.4424 - acc: 0.9036
Epoch 5/5
5988/5988 [==============================] - 1s 158us/step
Epoch 1/5
5988/5988 [==============================] - 4s 663us/step - loss: 1.6838 - acc: 0.6324
Epoch 2/5
5988/5988 [==============================] - 3s 551us/step - loss: 0.9311 - acc: 0.7894
Epoch 3/5
5988/5988 [==============================] - 3s 550us/step - loss: 0.6444 - acc: 0.8545
Epoch 4/5
5988/5988 [==============================] - 3s 534us/step - loss: 0.4603 - acc: 0.8981
Epoch 5/5
5988/5988 [==============================] - 1s 166us/step
Epoch 1/5
5988/5988 [==============================] - 8s 1ms/step - loss: 1.4405 - acc: 0.6860
Epoch 2/5
5988/5988 [==============================] - 7s 1ms/step - loss: 0.8326 - acc: 0.8136
Epoch 3/5
5988/5988 [==============================] - 7s 1ms/step - loss: 0.5761 - acc: 0.8699
Epoch 4/5
5988/5988 [==============================] - 7s 1ms/step - loss: 0.4206 - acc: 0.909

5988/5988 [==============================] - 8s 1ms/step - loss: 1.5874 - acc: 0.6520
Epoch 2/5
5988/5988 [==============================] - 5s 866us/step - loss: 0.6608 - acc: 0.8552
Epoch 3/5
5988/5988 [==============================] - 6s 975us/step - loss: 0.3152 - acc: 0.9335
Epoch 4/5
5988/5988 [==============================] - 6s 933us/step - loss: 0.1981 - acc: 0.9567
Epoch 5/5
5988/5988 [==============================] - 2s 330us/step
Epoch 1/5
5988/5988 [==============================] - 7s 1ms/step - loss: 1.5740 - acc: 0.6710
Epoch 2/5
5988/5988 [==============================] - 6s 1ms/step - loss: 0.6209 - acc: 0.8621
Epoch 3/5
5988/5988 [==============================] - 6s 1ms/step - loss: 0.3093 - acc: 0.9305
Epoch 4/5
5988/5988 [==============================] - 7s 1ms/step - loss: 0.1999 - acc: 0.9542
Epoch 5/5
5988/5988 [==============================] - 1s 242us/step
Epoch 1/5
5988/5988 [==============================] - 7s 1ms/step - loss: 1.5831 - acc: 0.6668
Ep

RandomizedSearchCV(cv=3, error_score='raise',
          estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000001943DA03208>,
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'batch_size': [10, 20, 30, 40, 50, 60, 70], 'optimizer': ['rmsprop', 'adam', 'adadelta'], 'keep_prob': array([0.1, 0.2, 0.3, 0.4, 0.5]), 'epochs': [5]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [153]:
print(search.best_params_)

{'optimizer': 'adam', 'keep_prob': 0.30000000000000004, 'epochs': 5, 'batch_size': 50}


In [158]:
# create model
model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu, input_shape=(10000, )))
model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(LABEL_DIMENSIONS, activation=tf.nn.softmax))

optimizer = tf.train.AdamOptimizer(learning_rate=0.001)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_29 (Dense)             (None, 64)                640064    
_________________________________________________________________
dense_30 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_31 (Dense)             (None, 46)                2990      
Total params: 647,214
Trainable params: 647,214
Non-trainable params: 0
_________________________________________________________________


In [159]:
batch_size = 50
epochs = 10

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.2)
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 7185 samples, validate on 1797 samples
Epoch 1/10
7185/7185 [==============================] - 4s 524us/step - loss: 1.7334 - acc: 0.6248 - val_loss: 1.1625 - val_acc: 0.7446
Epoch 2/10
7185/7185 [==============================] - 2s 291us/step - loss: 0.7969 - acc: 0.8227 - val_loss: 0.9353 - val_acc: 0.7924
Epoch 3/10
7185/7185 [==============================] - 2s 292us/step - loss: 0.4191 - acc: 0.9120 - val_loss: 0.9477 - val_acc: 0.7969
Epoch 4/10
7185/7185 [==============================] - 2s 311us/step - loss: 0.2655 - acc: 0.9454 - val_loss: 0.9549 - val_acc: 0.7991
Epoch 5/10
7185/7185 [==============================] - 2s 329us/step - loss: 0.1990 - acc: 0.9550 - val_loss: 0.9890 - val_acc: 0.7896
Epoch 6/10
7185/7185 [==============================] - 2s 304us/step - loss: 0.1697 - acc: 0.9584 - val_loss: 1.0124 - val_acc: 0.7913
Epoch 7/10
7185/7185 [==============================] - 2s 304us/step - loss: 0.1482 - acc: 0.9592 - val_loss: 1.0771 - val_acc: 0.7908
